In [1]:
!pip install transformers==4.28.0 datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-veapdr_3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-veapdr_3
  Resolved https://github.com/huggingface/accelerate to commit 330d60b817c7acb9f66e9fdcafec7efb391c0366
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding, pipeline
from datasets import load_dataset
import evaluate

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
data = load_dataset('csv', data_files = "cleaned_data.csv", encoding='utf8')

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
def preprocess_function(data):
    return tokenizer(data["cleaned_title"], truncation=True, padding='max_length')

In [7]:
data = data.map(preprocess_function, batched=True)

In [8]:
data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
data['label'] = data['train']['label'].type(torch.LongTensor) 
data_split = data['train'].train_test_split(test_size = 0.2, train_size = 0.8)
training_data = data_split['train']
validation_data = data_split['test']

In [10]:
accuracy = evaluate.load("accuracy")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [13]:
if torch.cuda.is_available:
    print("available")
    device = torch.device('cuda')
    model.cuda()
else:
    device = torch.device('cpu')    

available


In [16]:
training_args = TrainingArguments(
    output_dir="BERT_model",
    logging_steps=50,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=training_data,
    eval_dataset=validation_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/ungjus/BERT_model into local empty directory.


In [17]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.191600,0.143507,0.962249
2,0.066800,0.176109,0.965033
3,0.071300,0.186201,0.969599


TrainOutput(global_step=13470, training_loss=0.10338221247077718, metrics={'train_runtime': 5369.5121, 'train_samples_per_second': 20.068, 'train_steps_per_second': 2.509, 'total_flos': 1.4273892074778624e+16, 'train_loss': 0.10338221247077718, 'epoch': 3.0})

In [18]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/May17_04-39-27_12603438dc34/events.out.tfevents.1684298383.12603438dc34.27242.0:   0%|       …

To https://huggingface.co/ungjus/BERT_model
   ffdcb21..0727799  main -> main

   ffdcb21..0727799  main -> main

To https://huggingface.co/ungjus/BERT_model
   0727799..4d50e94  main -> main

   0727799..4d50e94  main -> main



'https://huggingface.co/ungjus/BERT_model/commit/0727799081c3cbb106aec9bc7844ddb5067f2a00'

In [9]:
text_classifier = pipeline(task="text-classification", model="ungjus/BERT_model")

In [13]:
bert_labels = text_classifier(validation_data['cleaned_title'])

In [16]:
y_hat = [0 if label['label'] == 'LABEL_0' else 1 for label in bert_labels]

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(validation_data['label'], y_hat)

0.9766146993318485

In [21]:
confusion_matrix(validation_data['label'], y_hat)

array([[4588,  127],
       [  83, 4182]])